In [2]:
from CU_Dataset_Factory import Hf_Loader, CU_Dataset_Factory

/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
factory = CU_Dataset_Factory('.')
train_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'train')
validation_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'validation')

In [4]:


train = factory.produce(train_l, 'tr_train.tsv', ['description'], 'label', 10, False)
test  = factory.produce(validation_l, 'tr_validation.tsv', ['description'], 'label', 10, False)

batch compute: 100%|██████████| 300/300 [00:00<00:00, 7799.97it/s, batch=30]


In [5]:
train_data = train[['label', 'description']].rename({'description': 'text'}, axis=1)
validation_data = test[['label', 'description']].rename({'description': 'text'}, axis=1)

In [6]:
train.head(5)

,description,wiki_name,qid,label
0,2012 film by M. Mohanan,916 (film),Q32786,1
1,American dance-punk band from California,!!!,Q371,2
2,Mort & Phil comic,¡Soborno!,Q3729947,2
3,American band,+44 (band),Q158611,2
4,"building in Monmouth, Wales","1 Monk Street, Monmouth",Q280375,1


In [7]:
from datasets import Dataset
from datasets import Features
from datasets import Split, Value

In [8]:
train_data = Dataset.from_pandas(train_data, features=Features({
    'label': Value('int32'),
    'text' : Value('string')
}), split=Split.TRAIN)

In [9]:
validation_data = Dataset.from_pandas(validation_data, features=Features({
    'label': Value('int32'),
    'text' : Value('string')
}), split=Split.TEST)



In [10]:
from transformers import AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [12]:
class Preprocessor:
    def __init__(self, tokenizer) -> None:
        self.tokenizer = tokenizer
    def process_samples(self, samples):
        return samples.map(lambda sample: self.tokenizer(sample['text'], truncation=True))

In [13]:
p = Preprocessor(tokenizer)

In [14]:
tokenize_train = p.process_samples(train_data)
tokenize_test = p.process_samples(validation_data)

Map: 100%|██████████| 300/300 [00:00<00:00, 2780.54 examples/s]


In [15]:
from transformers import DataCollatorWithPadding

In [16]:
collector = DataCollatorWithPadding(tokenizer)

In [17]:
import numpy as np
import evaluate

In [18]:
def compute_metrics(eval_pred):
    acc = evaluate.load('accuracy')
    
    
    pred, true = eval_pred
    pred = np.argmax(pred, axis=1)
    return acc.compute(predictions=pred, references=true)

In [19]:
cls2label = {0:'Cultural Agnostic', 1:'Cultural Rapresentative', 2:'Cultural Exclusive'}
label2cls = {l:c for c ,l in cls2label.items()}

In [20]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer


In [21]:
model = model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
traning_args = TrainingArguments(
    output_dir='CU_with_bert',
    eval_strategy='epoch',
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=traning_args,
    data_collator=collector,
    train_dataset=tokenize_train,
    eval_dataset=tokenize_test,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.673600,0.700462,0.713333
2,0.439200,0.751048,0.726667
3,0.293300,1.014506,0.723333


/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/andrea/minicon

TrainOutput(global_step=2346, training_loss=0.45589785563671376, metrics={'train_runtime': 1110.4948, 'train_samples_per_second': 16.887, 'train_steps_per_second': 2.113, 'total_flos': 97452983823318.0, 'train_loss': 0.45589785563671376, 'epoch': 3.0})